In [2]:
import pandas as pd 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
df = pd.read_csv('OnlineRetail.csv', encoding='ISO-8859-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
df['Description'] = df['Description'].str.strip()#to remove white spaces
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [4]:
#consolidate one transaction per row with each product
#looking only at sales for France to keep the dataset small
basket = (df[df['Country']=='France']
          .groupby(['InvoiceNo', 'Description'])
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
basket

C:\Users\Clarence\AppData\Local\Temp\ipykernel_13932\3340418254.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  basket = (df[df['Country']=='France']


Quantity                             \
Description 10 COLOUR SPACEBOY PEN 12 COLOURED PARTY BALLOONS   
InvoiceNo                                                       
536370                         0.0                        0.0   
536852                         0.0                        0.0   
536974                         0.0                        0.0   
537065                         0.0                        0.0   
537463                         0.0                        0.0   
...                            ...                        ...   
580986                         0.0                        0.0   
581001                         0.0                        0.0   
581171                         0.0                        0.0   
581279                         0.0                        0.0   
581587                         0.0                        0.0   

                                                                       \
Description 12 EGG HOUSE PAINTED WOOD 12 MESSAGE CARDS WITH ENVELOPES   
InvoiceNo                                                               
536370                            0.0                             0.0   
536852                            0.0                             0.0   
536974                            0.0                             0.0   
537065                            0.0                             0.0   
537463                            0.0                             0.0   
...                               ...                             ...   
580986                            0.0                             0.0   
581001                            0.0                             0.0   
581171                            0.0                             0.0   
581279                            0.0                             0.0   
581587                            0.0                             0.0   

                                                                               \
Description 12 PENCIL SMALL TUBE WOODLAND 12 PENCILS SMALL TUBE RED RETROSPOT   
InvoiceNo                                                                       
536370                                0.0                                 0.0   
536852                                0.0                                 0.0   
536974                                0.0                                 0.0   
537065                                0.0                                 0.0   
537463                                0.0                                 0.0   
...                                   ...                                 ...   
580986                                0.0                                 0.0   
581001                                0.0                                 0.0   
581171                                0.0                                 0.0   
581279                                0.0                                 0.0   
581587                                0.0                                 0.0   

                                                                   \
Description 12 PENCILS SMALL TUBE SKULL 12 PENCILS TALL TUBE POSY   
InvoiceNo                                                           
536370                              0.0                       0.0   
536852                              0.0                       0.0   
536974                              0.0                       0.0   
537065                              0.0                       0.0   
537463                              0.0                       0.0   
...                                 ...                       ...   
580986                              0.0                       0.0   
581001                              0.0                       0.0   
581171                              0.0                       0.0   
581279                              0.0                       0.0   
581587                              0.0                     

In [10]:
#Convert all positive number to one
#Convert all less than zero to 0
def encode_units(x):
    if x <= 0:
        return 0
    if x>= 1:
        return 1
basket_sets =basket.applymap(encode_units)#Apply a function to a Dataframe elementwise. This method applies a function that accepts and returns a scalar to every element of a DataFrame.
#basket_sets.drop('POSTAGE', inplace=True, axis=1) Error: POSTAGE, Solution: remove this line of code
basket_sets.dropna(inplace=True, axis=0) #Error: Found nan, Solution: This line of code
basket_sets

Quantity                             \
Description 10 COLOUR SPACEBOY PEN 12 COLOURED PARTY BALLOONS   
InvoiceNo                                                       
537967                           0                          0   
539113                           0                          0   
539435                           0                          0   
539688                           0                          0   
539727                           0                          0   
...                            ...                        ...   
580144                           0                          0   
580285                           0                          0   
580534                           0                          0   
580706                           0                          0   
581279                           0                          0   

                                                                       \
Description 12 EGG HOUSE PAINTED WOOD 12 MESSAGE CARDS WITH ENVELOPES   
InvoiceNo                                                               
537967                              0                               0   
539113                              0                               0   
539435                              0                               0   
539688                              0                               0   
539727                              0                               0   
...                               ...                             ...   
580144                              0                               0   
580285                              0                               0   
580534                              0                               0   
580706                              0                               0   
581279                              0                               0   

                                                                               \
Description 12 PENCIL SMALL TUBE WOODLAND 12 PENCILS SMALL TUBE RED RETROSPOT   
InvoiceNo                                                                       
537967                                  0                                   0   
539113                                  0                                   0   
539435                                  0                                   0   
539688                                  0                                   0   
539727                                  0                                   0   
...                                   ...                                 ...   
580144                                  0                                   0   
580285                                  0                                   0   
580534                                  0                                   0   
580706                                  0                                   0   
581279                                  0                                   0   

                                                                   \
Description 12 PENCILS SMALL TUBE SKULL 12 PENCILS TALL TUBE POSY   
InvoiceNo                                                           
537967                                0                         0   
539113                                0                         0   
539435                                0                         0   
539688                                0                         0   
539727                                0                         0   
...                                 ...                       ...   
580144                                0                         0   
580285                                0                         0   
580534                                0                         0   
580706                                0                         0   
581279                                0                     

In [6]:
#Generate frequent itemsets with support at least 7%
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules



C:\Users\Clarence\anaconda3\envs\Apriori\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"((Quantity, LUNCH BAG WOODLAND))","((Quantity, DOLLY GIRL LUNCH BOX))",0.116883,0.142857,0.090909,0.777778,5.444444,0.074212,3.857143
1,"((Quantity, DOLLY GIRL LUNCH BOX))","((Quantity, LUNCH BAG WOODLAND))",0.142857,0.116883,0.090909,0.636364,5.444444,0.074212,2.428571
2,"((Quantity, POSTAGE))","((Quantity, DOLLY GIRL LUNCH BOX))",0.675325,0.142857,0.116883,0.173077,1.211538,0.020408,1.036545
3,"((Quantity, DOLLY GIRL LUNCH BOX))","((Quantity, POSTAGE))",0.142857,0.675325,0.116883,0.818182,1.211538,0.020408,1.785714
4,"((Quantity, SPACEBOY LUNCH BOX))","((Quantity, DOLLY GIRL LUNCH BOX))",0.129870,0.142857,0.129870,1.000000,7.000000,0.111317,inf
...,...,...,...,...,...,...,...,...,...
529145,"((Quantity, SPACEBOY LUNCH BOX))","((UnitPrice, LUNCH BAG WOODLAND), (UnitPrice, ...",0.129870,0.077922,0.077922,0.600000,7.700000,0.067802,2.305195
529146,"((UnitPrice, SPACEBOY LUNCH BOX))","((UnitPrice, LUNCH BAG WOODLAND), (UnitPrice, ...",0.129870,0.077922,0.077922,0.600000,7.700000,0.067802,2.305195
529147,"((CustomerID, SPACEBOY LUNCH BOX))","((UnitPrice, LUNCH BAG WOODLAND), (UnitPrice, ...",0.129870,0.077922,0.077922,0.600000,7.700000,0.067802,2.305195
529148,"((Quantity, LUNCH BAG WOODLAND))","((UnitPrice, LUNCH BAG WOODLAND), (UnitPrice, ...",0.116883,0.077922,0.077922,0.666667,8.555556,0.068814,2.766234


In [7]:
#Another constraint
rules[(rules['lift'] >= 6) &
      (rules['confidence'] >= 0.8)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,"((Quantity, SPACEBOY LUNCH BOX))","((Quantity, DOLLY GIRL LUNCH BOX))",0.129870,0.142857,0.129870,1.000000,7.0,0.111317,inf
5,"((Quantity, DOLLY GIRL LUNCH BOX))","((Quantity, SPACEBOY LUNCH BOX))",0.142857,0.129870,0.129870,0.909091,7.0,0.111317,9.571429
6,"((UnitPrice, DOLLY GIRL LUNCH BOX))","((Quantity, DOLLY GIRL LUNCH BOX))",0.142857,0.142857,0.142857,1.000000,7.0,0.122449,inf
7,"((Quantity, DOLLY GIRL LUNCH BOX))","((UnitPrice, DOLLY GIRL LUNCH BOX))",0.142857,0.142857,0.142857,1.000000,7.0,0.122449,inf
12,"((UnitPrice, SPACEBOY LUNCH BOX))","((Quantity, DOLLY GIRL LUNCH BOX))",0.129870,0.142857,0.129870,1.000000,7.0,0.111317,inf
...,...,...,...,...,...,...,...,...,...
529126,"((CustomerID, DOLLY GIRL LUNCH BOX), (Quantity...","((UnitPrice, LUNCH BAG WOODLAND), (UnitPrice, ...",0.090909,0.077922,0.077922,0.857143,11.0,0.070838,6.454545
529130,"((Quantity, SPACEBOY LUNCH BOX), (Quantity, LU...","((UnitPrice, LUNCH BAG WOODLAND), (UnitPrice, ...",0.090909,0.077922,0.077922,0.857143,11.0,0.070838,6.454545
529133,"((UnitPrice, SPACEBOY LUNCH BOX), (Quantity, L...","((UnitPrice, LUNCH BAG WOODLAND), (UnitPrice, ...",0.090909,0.077922,0.077922,0.857143,11.0,0.070838,6.454545
529135,"((Quantity, LUNCH BAG WOODLAND), (CustomerID, ...","((UnitPrice, LUNCH BAG WOODLAND), (UnitPrice, ...",0.090909,0.077922,0.077922,0.857143,11.0,0.070838,6.454545


### Manual Apriori